In [ ]:
import tensorflow as tf

class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters, batch_normalization=True, conv_first=False):
    super(ResnetIdentityBlock, self).__init__(name='')
    
    self.residual_layers = []
    
    for i in range(len(filters)):
        
        if conv_first:
            setattr(self, 'conv' + str(i+1), tf.keras.layers.Conv2D(filters[i], kernel_size, padding='same'))
            self.residual_layers.append('conv' + str(i+1))

            if batch_normalization:
                setattr(self, 'bn' + str(i+1), tf.keras.layers.BatchNormalization())
                self.residual_layers.append('bn' + str(i+1))
        
        else:
            if batch_normalization:
                setattr(self, 'bn' + str(i+1), tf.keras.layers.BatchNormalization())
                self.residual_layers.append('bn' + str(i+1))
            
            setattr(self, 'conv' + str(i+1), tf.keras.layers.Conv2D(filters[i], kernel_size, padding='same'))
            self.residual_layers.append('conv' + str(i+1))

            
            
  def call(self, input_tensor, training=False):
    
    x = input_tensor
    
    for layer in self.residual_layers:
        if isinstance(layer, tf.keras.layers.Conv2D):
            x = getattr(self, layer)(x)
        else: 
            x = getattr(self, layer)(x, training=False)
        x = tf.nn.relu(x)
        
    x += input_tensor
    return tf.nn.relu(x)

In [ ]:
from keras.datasets import cifar10

(x_train,y_train),(x_test,y_test) = cifar10.load_data() 

x_train = x_train /255
x_test = x_test / 255

In [ ]:
from keras.utils.np_utils import to_categorical   

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=5,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.05)

In [ ]:
NUM_CLASSES = 10
INPUT_SHAPE = (32, 32, 3)

NUM_EPOCHS = 80
learning_rate = 1e-4
BATCH_SIZE=128

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=INPUT_SHAPE))

model.add(ResnetIdentityBlock((3,3), filters=(128, 128)))
model.add(ResnetIdentityBlock((3,3), filters=(128, 128)))
model.add(ResnetIdentityBlock((3,3), filters=(128, 128)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same'))

model.add(ResnetIdentityBlock((3,3), filters=(64, 64)))
model.add(ResnetIdentityBlock((3,3), filters=(64, 64)))
model.add(ResnetIdentityBlock((3,3), filters=(64, 64)))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.AveragePooling2D(pool_size=8))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))

In [ ]:
model.summary()

datagen.fit(x_train)

sgd = tf.keras.optimizers.Adam(learning_rate)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    validation_data=(x_test, y_test),
                    epochs=NUM_EPOCHS)

In [ ]:
history1 = model.history.history

# summarize history for accuracy
plt.plot(history1['accuracy'])
plt.plot(history1['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history1['loss'])
plt.plot(history1['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
mode = tf.keras.Sequential()
mode.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', input_shape=INPUT_SHAPE))

mode.add(ResnetIdentityBlock((3,3), filters=(256, 256)))
mode.add(ResnetIdentityBlock((3,3), filters=(256, 256)))
mode.add(ResnetIdentityBlock((3,3), filters=(256, 256)))
mode.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same'))


mode.add(ResnetIdentityBlock((3,3), filters=(128, 128)))
mode.add(ResnetIdentityBlock((3,3), filters=(128, 128)))
mode.add(ResnetIdentityBlock((3,3), filters=(128, 128)))
mode.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same'))

mode.add(ResnetIdentityBlock((3,3), filters=(64, 64)))
mode.add(ResnetIdentityBlock((3,3), filters=(64, 64)))
mode.add(ResnetIdentityBlock((3,3), filters=(64, 64)))

mode.add(tf.keras.layers.BatchNormalization())
mode.add(tf.keras.layers.Activation('relu'))
mode.add(tf.keras.layers.AveragePooling2D(pool_size=8))
mode.add(tf.keras.layers.Flatten())

mode.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))

In [ ]:
mode.summary()

datagen.fit(x_train)

sgd = tf.keras.optimizers.Adam(learning_rate)

mode.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

mode.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    validation_data=(x_test, y_test),
                    epochs=NUM_EPOCHS)

In [ ]:
history = mode.history.history

# summarize history for accuracy
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#VGG16 MODEL

In [ ]:
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow.keras as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.utils.np_utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

vgg16_model = VGG16(weights='imagenet',
                    include_top=False, 
                    classes=10,
                    input_shape=(32,32,3)# input: 32x32 images with 3 channels -> (32, 32, 3) tensors.
                   )

In [ ]:
model1 = Sequential()
for layer in vgg16_model.layers:
    model1.add(layer)


In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
model1.add(Flatten())
model1.add(Dense(512, activation='relu', name='hidden1'))
model1.add(Dropout(0.4))
model1.add(Dense(256, activation='relu', name='hidden2'))
model1.add(Dropout(0.4))
model1.add(Dense(10, activation='softmax', name='predictions'))

model1.summary()


In [ ]:
sgd = optimizers.SGD(lr=0.001, momentum=0.9)

# For a multi-class classification problem
model1.compile(loss='categorical_crossentropy',optimizer= sgd,metrics=['accuracy'])


es = EarlyStopping(patience=10, monitor='val_accuracy', mode='max')

# initialize the number of epochs and batch size
EPOCHS = 100
BS = 32

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                        horizontal_flip=True, fill_mode="nearest")
 
# train the model
history = model1.fit_generator(aug.flow(x_train,y_train, batch_size=BS),validation_data=(x_test,y_test), 
                              steps_per_epoch=len(x_train) // BS ,epochs=EPOCHS ,callbacks=[es])


In [ ]:


# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()